In [1]:
# import pandas as  pd

# df = pd.read_csv("/data/songlei/maro/examples/supply_chain/results/SCI_5SKUs_DIST_DQN/output_product_metrics.csv")
# df[((df['name'] == "FOODS_3_222") & (df['facility_name'] == 'CA_1'))][['tick', 'consumer_order_product_cost']].head(100)

In [2]:
import numpy as np
import pandas as pd


num_products_to_sample = 50
item = pd.read_parquet("data/item_bak.parquet")
bom_raw = pd.read_parquet("data/billofmateriallines_bak.parquet")
orderline = pd.read_parquet("data/orderline_bak.parquet")
production_assignment = pd.read_parquet("data/production_assignment_bak.parquet")
distribution_node = pd.read_parquet("data/distribution_node_bak.parquet")
production_plant = pd.read_parquet("data/production_plant_bak.parquet")
routes = pd.read_parquet("data/routes_bak.parquet")
storage_assignment = pd.read_parquet("data/storage_assignment_bak.parquet")
warehouse = pd.read_parquet("data/warehouse_bak.parquet")


In [3]:
storage_assignment.loc[:, "WarehouseId"] = storage_assignment['WarehouseId'].map(lambda x: x.replace("WH_", ""))
storage_assignment[storage_assignment["ItemId"] == "FOODS_1_055"]

,ItemId,WarehouseId,UnitPrice,StorageCost,UnitOfStorageCost,InventoryOnHand
3988,FOODS_1_055,STRG_CA_1,2.41,0.00165,daily,125.45
3989,FOODS_1_055,STRG_TX_1,2.41,0.00082,daily,65.92
3990,FOODS_1_055,STRG_WI_1,2.41,0.00097,daily,55.55
7273,FOODS_1_055,CA_1,3.90,0.00111,daily,21.33
7274,FOODS_1_055,CA_2,3.90,0.00131,daily,46.51
7275,FOODS_1_055,CA_3,3.92,0.00241,daily,44.45
7276,FOODS_1_055,CA_4,3.88,0.00221,daily,13.16
7277,FOODS_1_055,TX_1,3.92,0.00116,daily,21.78
7278,FOODS_1_055,TX_2,3.89,0.00243,daily,21.38
7279,FOODS_1_055,TX_3,3.89,0.00177,daily,22.76


In [4]:
finished_good_list = item[item['IsItemTypeFinishedGood']]['ItemId'].unique().tolist()
if len(finished_good_list) > num_products_to_sample:
    sampled_good_list = np.random.choice(finished_good_list, size=num_products_to_sample, replace=False).tolist()
else:
    sampled_good_list = finished_good_list
orderline[orderline["ItemId"].isin(sampled_good_list)].to_parquet("data/orderline.parquet", index=False)
production_assignment[production_assignment['ItemId'].isin(sampled_good_list)].to_parquet("data/production_assignment.parquet", index=False)


In [5]:
storage_info_dict = {}
component_info = {}
for _, row in storage_assignment.iterrows():
    if row['WarehouseId'] not in storage_info_dict:
        storage_info_dict[row['WarehouseId']] = {}
    storage_info_dict[row['WarehouseId']][row['ItemId']] = {'price': row["UnitPrice"], 'storage_cost': row["StorageCost"], 'init_stock': int(row["InventoryOnHand"])}
    if row["WarehouseId"].startswith("STRG_CA_") or row["WarehouseId"].startswith("PLANT_"):
        component_info[row['ItemId']] = {'price': row["UnitPrice"], 'storage_cost': row["StorageCost"], 'init_stock': int(row["InventoryOnHand"])}

In [6]:
bom_raw = bom_raw[((bom_raw["ItemId"].isin(sampled_good_list)) & (bom_raw["ComponentQuantity"] > 0))]
bom_raw = bom_raw.groupby("ItemId").head(2).reset_index()
bom_raw.to_parquet("data/billofmateriallines.parquet", index=False)
material_sku_list = bom_raw['ComponentItemId'].unique().tolist()
item[item['ItemId'].isin(material_sku_list+sampled_good_list)].to_parquet("data/item.parquet", index=False)

routes = pd.read_parquet("data/routes_bak.parquet")
routes.loc[:, "StartingLocationId"] = routes["StartingLocationId"].map(lambda x: x.replace("WH_", ''))
routes.loc[:, "EndingLocationId"] = routes["EndingLocationId"].map(lambda x: x.replace("WH_", ''))
routes.to_parquet("data/routes.parquet")

warehouse.loc[:, "WarehouseId"] = warehouse['WarehouseId'].map(lambda x: x.replace("WH_", ""))
warehouse.to_parquet("data/warehouse.parquet", index=False)

distribution_node.loc[:, "FacilityId"] = distribution_node['FacilityId'].map(lambda x: x.replace("WH_", ""))
distribution_node.to_parquet("data/distribution_node.parquet")

In [7]:
# Distribution 
routes = pd.read_parquet("data/routes.parquet")
distribution_config = """
vendor_distribution: &vendor_distribution
  config:
    train:
      number: 10000
"""
for facility_id, routes_info in routes.groupby("StartingLocationId"):
  distribution_config += f"""
{facility_id}_distribution: &{facility_id}_distribution
  config:
"""
  for _, route in routes_info.iterrows():
    distribution_config += f"""
    {route['RouteId']}:
      number: 10000
"""



In [8]:
"""
vendor_storage: &vendor_storage
  config:
    capacity: 1000000000
    unit_storage_cost: 0.1

plant_storage: &plant_storage
  config:
    -
      id: 1
      capacity: 1000000000
      unit_storage_cost: 0.1
    -
      id: 2
      capacity: 1000000000
      unit_storage_cost: 0.1

warehouse_storage: &warehouse_storage
  config:
    capacity: 1000000000
    unit_storage_cost: 0.1

store_storage: &store_storage
  config:
    capacity: 1000000000
    unit_storage_cost: 0.1
"""

distribution_node = pd.read_parquet("data/distribution_node.parquet")
order_cost_dict = {row["FacilityId"]: row["OrderCost"] for _, row in distribution_node.iterrows()}

warehouse = pd.read_parquet("data/warehouse.parquet")
storage_config = """
vendor_storage: &vendor_storage
  config:
    capacity: 1000000000
    unit_storage_cost: 0.1
"""
for _, row in warehouse.iterrows():
    storage_config += f"""
{row['WarehouseId']}_storage: &{row['WarehouseId']}_storage
  config:
    capacity: {int(row['StorageCapacity'])}
    unit_storage_cost: 0.1
"""


In [9]:
import pandas as pd 

# SKU configuration
"""
    - id: 10
      name: "component_1"
    - id: 11
      name: "component_2"
    - id: 20
      name: "food_1"
    - id: 30
      name: "hobby_1"
      output_units_per_lot: 1
      bom:
        # The key could be sku id or sku name.
        "component_1": 1
        "component_2": 1
"""

facility_max_price = {}

sku_def_config = ""
cur_id = 10
bom_raw = pd.read_parquet("data/billofmateriallines.parquet")
item = pd.read_parquet("data/item.parquet")

sku_wot_bom = item[(item['IsItemTypeFinishedGood'] & ~item['VendorId'].isnull())]['ItemId'].unique().tolist() + bom_raw["ComponentItemId"].unique().tolist()

for sku in sku_wot_bom:
  sku_def_config += f"""
    - id: {cur_id}
      name: "{sku}"
  """
  cur_id += 1

bom = bom_raw[['ItemId', 'ComponentItemId', 'ComponentQuantity']].groupby("ItemId")
for key, val in bom:
  sku_def_config += f"""
    - id: {cur_id}
      name: "{key}"
      output_units_per_lot: 1
      bom:
  """
  cur_id += 1
  for _, row in val.iterrows():
    sku_def_config += f"""
        "{row['ComponentItemId']}": {row['ComponentQuantity']}
    """
print(cur_id)

70


In [10]:
"""
    - name: "Vendor_001"
      definition_ref: "VendorFacility"
      skus:
        "food_1":
          price: 60
          init_stock: 1000
          sub_storage_id: 0
          has_manufacture: True
          max_manufacture_rate: 100
          manufacture_leading_time: 1
          unit_product_cost: 14
          unit_delay_order_penalty: 1000
      ...
      children:
        storage: *single_storage
        distribution: *mix_distribution
      config:
        unit_delay_order_penalty: 1000
        unit_order_cost: 10
"""
import random 

orderline = pd.read_parquet("data/orderline.parquet")
# product_cost_dict = {row["ItemId"]: row["ItemUnitCost"] for _, row in orderline[["ItemId", "ItemUnitCost"]].groupby("ItemId").mean().reset_index().iterrows()}
vendor_config = ""
for vendor_id in item[~item['VendorId'].isnull()]["VendorId"].unique():
  vendor_config += f"""
    - name: "{vendor_id}"
      definition_ref: "VendorFacility"
      skus:
  """
  vendor_sku = item[item['VendorId'] == vendor_id]
  for _, row in vendor_sku.iterrows():
    product_id = row['ItemId']
    prc = component_info[product_id]['price']
    facility_max_price[vendor_id] = max(facility_max_price.get(vendor_id, 0), prc)
    vendor_config += f"""
        "{product_id}":
          price: {prc}
          init_stock: {random.randint(10000, 20000)}
          sub_storage_id: 0
          has_manufacture: True
          max_manufacture_rate: 10000
          manufacture_leading_time: 1
          unit_product_cost: {prc}
          unit_delay_order_penalty: 0
    """
  vendor_config += f"""
      children:
        storage: *vendor_storage
        distribution: *vendor_distribution
      config:
        unit_delay_order_penalty: 0
        unit_order_cost: 1.0
  """
  


In [11]:
production_assignment = pd.read_parquet("data/production_assignment.parquet")
production_assignment.head()

,ItemId,ProductionPlantId,ProductionAssignmentId,ManufactureLeadTime,ManufactureRate,UnitMaterialCost,UnitManufactureCost,UnitPrice
0,HOBBIES_2_054,PLANT_0,ProdAssign_12,3,17.65,394.56,523.54,918.10
1,HOBBIES_1_251,PLANT_1,ProdAssign_26,1,18.22,3703.64,7329.27,11032.91
2,HOBBIES_2_070,PLANT_1,ProdAssign_32,1,19.47,25.77,13.58,39.35
3,HOBBIES_1_122,PLANT_0,ProdAssign_43,5,30.46,207.62,180.47,388.09
4,HOBBIES_1_206,PLANT_0,ProdAssign_73,3,23.58,38.58,26.82,65.40


In [12]:
production_assignment[production_assignment["ProductionPlantId"]=="PLANT_0"]

,ItemId,ProductionPlantId,ProductionAssignmentId,ManufactureLeadTime,ManufactureRate,UnitMaterialCost,UnitManufactureCost,UnitPrice
0,HOBBIES_2_054,PLANT_0,ProdAssign_12,3,17.65,394.56,523.54,918.10
3,HOBBIES_1_122,PLANT_0,ProdAssign_43,5,30.46,207.62,180.47,388.09
4,HOBBIES_1_206,PLANT_0,ProdAssign_73,3,23.58,38.58,26.82,65.40


In [13]:
"""
    - name: "Plant_001"
      definition_ref: "PlantFacility"
      skus:
        "component_1":
          price: 5
          init_stock: 3000
          sub_storage_id: 1
          has_consumer: True
          unit_order_cost: 5
        "hobby_1":
          price: 70
          init_stock: 2500
          sub_storage_id: 2
          has_manufacture: True
          max_manufacture_rate: 100
          manufacture_leading_time: 2
          unit_product_cost: 35
      children:
        storage: *multi_storage
        distribution: *simple_distribution
      config:
        unit_delay_order_penalty: 1000
        unit_order_cost: 10

"""
production_assignment = pd.read_parquet("data/production_assignment.parquet")

plant_config = ""
for plant_id in production_assignment["ProductionPlantId"].unique():
  plant_sku_list = production_assignment[production_assignment["ProductionPlantId"]==plant_id]["ItemId"].unique().tolist()
  plant_config += f"""
    - name: "{plant_id}"
      definition_ref: "PlantFacility"
      skus:
  """
  for _, row in production_assignment[production_assignment["ProductionPlantId"]==plant_id].iterrows():
    product_id = row['ItemId']
    prc = storage_info_dict[plant_id][product_id]['price']
    facility_max_price[plant_id] = max(facility_max_price.get(plant_id, 0), prc)
    plant_config += f"""
        "{row['ItemId']}":
          price: {prc}
          init_stock: {storage_info_dict[plant_id][product_id]['init_stock']}
          sub_storage_id: 0
          has_manufacture: True
          max_manufacture_rate: {row['ManufactureRate']*100}
          manufacture_leading_time: {row['ManufactureLeadTime']}
          unit_product_cost: {row['UnitManufactureCost']+row['UnitMaterialCost']}
          unit_storage_cost: {storage_info_dict[plant_id][product_id]['storage_cost']}
          unit_delay_order_penalty: 0
  """
  material_sku_list = bom_raw[bom_raw['ItemId'].isin(plant_sku_list)]['ComponentItemId'].unique().tolist()
  for material_sku in material_sku_list:
    prc = storage_info_dict[plant_id][material_sku]['price']
    facility_max_price[plant_id] = max(facility_max_price.get(plant_id, 0), prc)
    plant_config += f"""
        "{material_sku}":
          price: {prc}
          init_stock: {storage_info_dict[plant_id][material_sku]['init_stock']}
          unit_storage_cost: {storage_info_dict[plant_id][material_sku]['storage_cost']}
          sub_storage_id: 0
          has_consumer: True
          unit_product_cost: {prc}
          unit_delay_order_penalty: 0
  """
  plant_config += f"""
      children:
        storage: *{plant_id}_storage
        distribution: *{plant_id}_distribution
      config:
        unit_delay_order_penalty: 0
        unit_order_cost: {order_cost_dict[plant_id]}
"""

In [14]:
# warehouse_config
"""   
    - name: "Warehouse_001"
      definition_ref: "WarehouseFacility"
      skus:
        "food_1":
          price: 60
          init_stock: 2000
          sub_storage_id: 0
          has_consumer: True
        "hobby_1":
          price: 70
          init_stock: 2000
          sub_storage_id: 0
          has_consumer: True
      children:
        storage: *single_storage
        distribution: *mix_distribution
      config:
        unit_delay_order_penalty: 0
        unit_order_cost: 10
"""
warehouse_config = ""

for warehouse_id in ["STRG_CA_1", "STRG_TX_1", "STRG_WI_1"]:
  warehouse_config += f"""
    - name: "{warehouse_id}"
      definition_ref: "WarehouseFacility"
      skus:
  """
  sku_num = item[item["IsItemTypeFinishedGood"]]["ItemId"].unique().shape[0]
  for item_id in item[item["IsItemTypeFinishedGood"]]["ItemId"].unique():
    prc = storage_info_dict[warehouse_id][item_id]['price']
    facility_max_price[warehouse_id] = max(facility_max_price.get(warehouse_id, 0), prc)
    warehouse_config += f"""
        "{item_id}":
          price: {prc}
          init_stock: {storage_info_dict[warehouse_id][item_id]['init_stock']}
          unit_storage_cost: {storage_info_dict[warehouse_id][item_id]['storage_cost']}
          sub_storage_id: 0
          has_consumer: True
    """
  warehouse_config += f"""
      children:
        storage: *{warehouse_id}_storage
        distribution: *{warehouse_id}_distribution
      config:
        unit_delay_order_penalty: 0
        unit_order_cost: {order_cost_dict[warehouse_id]}
  """

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import os
os.system("rm data/*preprocessed.csv")


orderline = pd.read_parquet("data/orderline.parquet")
orderline.loc[:, "ConfirmedDeliveryDate"] = pd.to_datetime(orderline["ConfirmedDeliveryDate"])
item = pd.read_parquet("data/item.parquet")


start_dt, end_dt = orderline["ConfirmedDeliveryDate"].min(), orderline["ConfirmedDeliveryDate"].max()
total_num_days = (end_dt - start_dt).days + 1
item_id_list = item[item["IsItemTypeFinishedGood"]]["ItemId"].unique().tolist()
item_cost_dict = {row["ItemId"]: row["ItemUnitCost"] for _, row in orderline[["ItemId", "ItemUnitCost"]].groupby("ItemId").mean().reset_index().iterrows()}
df_pivot_list = []
for item_id in item_id_list:
    # filled_sales = [0]*total_num_days
    filled_sales = [random.randint(1, 5) for _ in range(total_num_days)]
    filled_prc = [storage_info_dict['CA_1'][item_id]['price']]*total_num_days
    # filled_prc = [item_cost_dict[item_id] + 100.0]*total_num_days
    df_pivot_list.append(pd.DataFrame(data={'ItemId': [item_id]*total_num_days,
                                            'Date': pd.date_range(start_dt, end_dt),
                                            'filled_sales': filled_sales,
                                            'filled_prc': filled_prc}))
df_pivot = pd.concat(df_pivot_list)
df_pivot.head()

df_sales = (orderline[["WarehouseId", "ItemId", "ConfirmedDeliveryDate", "OrderLineQuantity", "ItemUnitCost"]]
            .groupby(["WarehouseId", "ItemId", "ConfirmedDeliveryDate"])
            .agg({"OrderLineQuantity": np.sum, "ItemUnitCost": np.mean})
            .reset_index()
            )
df_sales.columns = ["StoreId", "ItemId", "Date", "Sales", "Cost"]
for store_id in df_sales["StoreId"].unique().tolist():
    df_store_sales = df_sales[df_sales['StoreId'] == store_id]
    df_store_sales = pd.merge(df_pivot, df_store_sales, 
                              left_on=["ItemId", "Date"], 
                              right_on=["ItemId", "Date"],
                              how='left')
    df_store_sales["Sales"].fillna(df_store_sales["filled_sales"], inplace=True)
    df_store_sales["Cost"].fillna(df_store_sales["filled_prc"], inplace=True)
    df_store_sales.loc[:, "Price"] = df_store_sales["Cost"].map(lambda x: x)
    df_store_sales["StoreId"].fillna(store_id, inplace=True)
    df_store_sales.loc[:, "Sales"] = df_store_sales['Sales'].map(lambda x: int(x))
    df_store_sales[["ItemId", "Date", "Price", "Cost", "Sales"]].to_csv(f"data/store_{store_id}.csv", index=False)

In [16]:
# store config
"""
    - name: "Store_001"
      definition_ref: "StoreFacility"
      skus:
        "food_1":
          price: 150
          init_stock: 500
          sub_storage_id: 0
          has_consumer: True
          has_seller: True
          sale_gamma: 60
          backlog_ratio: 0.1
      children:
        storage: *single_storage
      config:
        unit_order_cost: 20
        dynamics_sampler_type: processed_price_demand # What kind of sampler to use.
        sku_column: "SKU" # SKU column name
        price_column: "Price" # Price column name
        demand_column: "Sales" # Demand column name
        datetime_column: "Date" # Date-time column name
        file_path: "/data/songlei/maro/maro/simulator/scenarios/supply_chain/topologies/plant/sample.csv"
"""
store = pd.read_parquet("data/warehouse.parquet")
store_config = ""
for store_id in store['WarehouseId'].unique():
  if store_id[:2] not in ["CA", "TX", "WI"]:
    continue
  store_config += f"""
    - name: "{store_id}"
      definition_ref: "StoreFacility"
      skus:
  """
  df_store_sales = pd.read_csv(f"data/store_{store_id}.csv")
  sale_mean_dict = {row["ItemId"]: row["Sales"] for _,row in df_store_sales[['ItemId', "Sales"]].groupby("ItemId").mean().reset_index().iterrows()}
  for item_id in item[item["IsItemTypeFinishedGood"]]["ItemId"].unique():
    prc = storage_info_dict[store_id][item_id]['price']
    facility_max_price[store_id] = max(facility_max_price.get(store_id, 0), prc)
    store_config += f"""
        "{item_id}":
          price: {prc}
          init_stock: {storage_info_dict[store_id][item_id]['init_stock']}
          unit_storage_cost: {storage_info_dict[store_id][item_id]['storage_cost']}
          sub_storage_id: 0
          has_consumer: True
          has_seller: True
          sale_gamma: {sale_mean_dict.get(item_id, 1.0)}
          backlog_ratio: 0.1
    """
  store_config += f"""
      children:
        storage: *{store_id}_storage
        distribution: *{store_id}_distribution
      config:
        unit_order_cost: {order_cost_dict[store_id]}
        file_path: "/data/songlei/maro/examples/supply_chain/data/store_{store_id}.csv"
        unit_delay_order_penalty: 0
  """


In [17]:
"""
    "Store_001":
      "food_1":
        "Warehouse_001":
          "air":
            vlt: 2
            cost: 3.0
          "train":
            vlt: 7
            cost: 0.5
    "Warehouse_001":
      "food_1":
        "Vendor_001":
          "air":
            vlt: 2
            cost: 2.0
          "train":
            vlt: 7
            cost: 0.3
      "hobby_1":
        "Plant_001":
          "train":
            vlt: 7
            cost: 0.35
    "Plant_001":
      "component_1":
        "Vendor_001":
          "train":
            vlt: 7
            cost: 0.2
settings:
  pending_order_len: 4  # For initialization of Consumer Units
  start_date_time: "2021-01-01"
"""

# topology config
default_vendor_config = {}
topology_config = ""
for store_id in store['WarehouseId'].unique():
  if store_id[:2] not in ["CA", "TX", "WI"]:
    continue
  default_vendor_config[store_id] = {}
  store_route = routes[routes['EndingLocationId'] == store_id]
  topology_config += f"""
    "{store_id}":
  """
  for item_id in item[item["IsItemTypeFinishedGood"]]["ItemId"].unique():
    topology_config += f"""
      "{item_id}":
      """
    for store_vendor, vendor_route in store_route.groupby("StartingLocationId"):
      topology_config += f"""
        "{store_vendor}":
      """
      for _, _route in vendor_route.iterrows():
        if ((store_vendor.startswith("STRG_"))
            and (store_id[:2] in store_vendor) 
            and (item_id not in default_vendor_config[store_id].keys())):
          default_vendor_config[store_id][item_id] = _route["RouteId"]
        topology_config += f"""
          "{_route["RouteId"]}":
            vlt: {_route["Duration"]}
            cost: {_route["TransportationCost"]/(_route["Duration"]+1)}
      """
production_assignment = pd.read_parquet("data/production_assignment.parquet")
hobbies_plant_dict = {row["ItemId"]: row["ProductionPlantId"] for _, row in production_assignment.iterrows()}
non_hobbies_vendor_dict = {row['ItemId']: row["VendorId"] for _, row in item[~item['VendorId'].isnull()].iterrows()}
for warehouse_id in ["STRG_CA_1", "STRG_TX_1", "STRG_WI_1"]:
  topology_config += f"""
    "{warehouse_id}":
  """
  default_vendor_config[warehouse_id] = {}
  for item_id in item[item["IsItemTypeFinishedGood"]]["ItemId"].unique():
    if item_id.startswith("HOBBIES"):
      topology_config += f"""
      "{item_id}":
      """
      vendor_id = hobbies_plant_dict[item_id]
      warehouse_route = routes[((routes['EndingLocationId'] == warehouse_id) & (routes['StartingLocationId'] == vendor_id))]
      topology_config += f"""
        "{vendor_id}":
      """
      for _, _route in warehouse_route.iterrows():
        if item_id not in default_vendor_config[warehouse_id].keys():
          default_vendor_config[warehouse_id][item_id] = _route["RouteId"]
        topology_config += f"""
          "{_route["RouteId"]}":
            vlt: {_route["Duration"]}
            cost: {_route["TransportationCost"]/(_route["Duration"]+1)}
      """
    else:
      vendor_id = non_hobbies_vendor_dict[item_id]
      default_vendor_config[warehouse_id][item_id] = "train"
      topology_config += f"""
      "{item_id}":
        "{vendor_id}":
          "train":
            vlt: 1
            cost: 0.3
      """
for plant_id in production_assignment['ProductionPlantId'].unique():
  plant_sku = production_assignment[production_assignment["ProductionPlantId"] == plant_id]
  material_sku = bom_raw[bom_raw['ItemId'].isin(plant_sku["ItemId"])]
  default_vendor_config[plant_id] = {}
  topology_config += f"""
    "{plant_id}":
  """
  for _, row in material_sku.iterrows():
    item_id = row['ComponentItemId']
    vendor_id = non_hobbies_vendor_dict[item_id]
    default_vendor_config[plant_id][item_id] = 'train'
    topology_config += f"""
      "{item_id}":
        "{vendor_id}":
          "train":
            vlt: 1
            cost: 0.3
    """
setting_config = f"""
  pending_order_len: 4
  start_date_time: "{start_dt.strftime("%Y-%m-%d")}"
"""

In [18]:
"""
facilities:
    Vendor_001:
      max_price: 33
    Plant_001:
      max_price: 33
    Warehouse_001:
      max_price: 33
    Store_001:
      max_price: 33
"""
policy_parameter = """
  facilities:
"""
for key, val in facility_max_price.items():
  policy_parameter += f"""
    {key}:
      max_price: {val}
  """

In [19]:
config = f"""
facility_definitions:
  # Key: would be used as "definition_ref" in world.facilities
  VendorFacility:
    # class: the class alias defined in core.yml
    class: "SupplierFacility"
    datamodel: "FacilityDataModel"
    children:
      storage:
        class: "StorageUnit"
      distribution:
        class: "DistributionUnit"
    products:
      class: "ProductUnit"
      config:
        manufacture:
          class: "SimpleManufactureUnit"
  PlantFacility:
    class: "SupplierFacility"
    datamodel: "FacilityDataModel"
    children:
      storage:
        class: "StorageUnit"
      distribution:
        class: "DistributionUnit"
    products:
      class: "ProductUnit"
      config:
        manufacture:
          class: "ManufactureUnit"
        consumer:
          class: "ConsumerUnit"
  WarehouseFacility:
    class: "WarehouseFacility"
    children:
      storage:
        class: "StorageUnit"
      distribution:
        class: "DistributionUnit"
    products:
      class: "ProductUnit"
      config:
        consumer:
          class: "ConsumerUnit"
  StoreFacility:
    class: "OuterRetailerFacility"
    datamodel: "FacilityDataModel"
    children:
      storage:
        class: "StorageUnit"
      distribution:
        class: "DistributionUnit"
    products:
      class: "StoreProductUnit"
      config:
        consumer:
          class: "ConsumerUnit"
        seller:
          class: "OuterSellerUnit"
          config:
            sale_hist_len: 14
    config:
      dynamics_sampler_type: processed_price_demand # What kind of sampler to use.
      sku_column: "ItemId" # SKU column name
      price_column: "Price" # Price column name
      demand_column: "Sales" # Demand column name
      datetime_column: "Date" # Date-time column name
      file_path: "/path/to/data.csv" # full path to data file, override by each store instance

{distribution_config}
{storage_config}
world:
  skus:
{sku_def_config}
  facilities:
{vendor_config}
{plant_config}
{warehouse_config}
{store_config}
  topology:
{topology_config}
settings:
{setting_config}
policy_parameter:
{policy_parameter}
"""

import os

os.makedirs("/data/songlei/maro/maro/simulator/scenarios/supply_chain/topologies/SCI/", exist_ok=True)
with open("/data/songlei/maro/maro/simulator/scenarios/supply_chain/topologies/SCI/config.yml", "w") as f:
    f.write(config)


In [20]:
with open("rl/default_vendor_config.py", "w") as f:
    f.write("default_vendor = " + str(default_vendor_config))